In [1]:
import tensorflow as tf

#Importing the Dataset
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteY171X6/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteY171X6/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [2]:
#Getting the training and testing sets
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [3]:
training_labels_final

array([0, 0, 0, ..., 0, 0, 1])

In [5]:
#Tokenization and Padding

#All hyperparameters
vocab_size = 10000  #Num of words for tokenizer
embedding_dim = 16  #Embedding dimension
max_length = 120    #Padding length
trunc_type='post'   #Type of truncating
oov_tok = "<OOV>"   #Out of vocabulary

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length,truncating=trunc_type)

In [10]:
#Defining the neural network that will perform word embedding
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length), #Incoming is T X 16
    #tf.keras.layers.Flatten(),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 120, 16)           160000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_4 (Dense)             (None, 6)                 102       
                                                                 
 dense_5 (Dense)             (None, 1)                 7         
                                                                 
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________


In [11]:
#Training
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 8s 9ms/step - loss: 0.5699 - accuracy: 0.7293 - val_loss: 0.4222 - val_accuracy: 0.8186
Epoch 2/10
782/782 [==============================] - 7s 10ms/step - loss: 0.3404 - accuracy: 0.8594 - val_loss: 0.3704 - val_accuracy: 0.8344
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2769 - accuracy: 0.8871 - val_loss: 0.3700 - val_accuracy: 0.8360
Epoch 4/10
782/782 [==============================] - 7s 9ms/step - loss: 0.2409 - accuracy: 0.9042 - val_loss: 0.3831 - val_accuracy: 0.8348
Epoch 5/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2152 - accuracy: 0.9191 - val_loss: 0.4072 - val_accuracy: 0.8264
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 0.1939 - accuracy: 0.9281 - val_loss: 0.4306 - val_accuracy: 0.8271
Epoch 7/10
782/782 [==============================] - 8s 10ms/step - loss: 0.1767 - accuracy: 0.9368 - val_loss: 0.4676 - val_accuracy: 0.8158
E

In [12]:
#Printing the weights of the embedding layer
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) #shape: (vocab_size, embedding_dim)

(10000, 16)
